## Setup

Run the cells below to setup and install the required libraries. For our experiment we will need `accelerate`, `peft`, `transformers`, `datasets` and TRL to leverage the recent [`SFTTrainer`](https://huggingface.co/docs/trl/main/en/sft_trainer). We will use `bitsandbytes` to [quantize the base model into 4bit](https://huggingface.co/blog/4bit-transformers-bitsandbytes). We will also install `einops` as it is a requirement to load Falcon models.

In [ ]:
!pip install -q -U trl transformers accelerate git+https://github.com/huggingface/peft.git
!pip install -q datasets bitsandbytes einops wandb

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.9/133.9 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 34.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.9/100.9 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 77.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252

## Dataset



In [ ]:
import json
from pathlib import Path
from pprint import pprint


file_path="/content/denemeindir.json"
data = json.loads(Path(file_path).read_text())

In [ ]:
from datasets import load_dataset, Dataset

issues_dataset = load_dataset("json", data_files="/content/deneme.json", split="train")
issues_dataset

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['text'],
    num_rows: 118
})

In [ ]:
from datasets import load_dataset, Dataset

#dataset_name = "timdettmers/openassistant-guanaco" ###Human ,.,,,,,, ###Assistant

dataset_name = 'databricks/databricks-dolly-15k' #french novels
dataset = load_dataset(dataset_name, split="train")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
dataset

Dataset({
    features: ['instruction', 'context', 'response', 'category'],
    num_rows: 15011
})

In [ ]:
dataset[1]

{'instruction': 'Which is a species of fish? Tope or Rope',
 'context': '',
 'response': 'Tope',
 'category': 'classification'}

## Loading the model

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer

model_name = "meta-llama/Llama-2-7b"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True
)
model.config.use_cache = False

config.json:   0%|          | 0.00/626 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

pytorch_model-00001-of-00014.bin:   0%|          | 0.00/981M [00:00<?, ?B/s]

pytorch_model-00002-of-00014.bin:   0%|          | 0.00/967M [00:00<?, ?B/s]

pytorch_model-00003-of-00014.bin:   0%|          | 0.00/967M [00:00<?, ?B/s]

...

pytorch_model-00012-of-00014.bin:   0%|          | 0.00/967M [00:00<?, ?B/s]

pytorch_model-00013-of-00014.bin:   0%|          | 0.00/967M [00:00<?, ?B/s]

pytorch_model-00014-of-00014.bin:   0%|          | 0.00/847M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/14 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

Let's also load the tokenizer below

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/676 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

In [ ]:
from peft import LoraConfig, get_peft_model

lora_alpha = 16
lora_dropout = 0.1
lora_r = 64

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM"
)

## Loading the trainer

Here we will use the [`SFTTrainer` from TRL library](https://huggingface.co/docs/trl/main/en/sft_trainer) that gives a wrapper around transformers `Trainer` to easily fine-tune models on instruction based datasets using PEFT adapters. Let's first load the training arguments below.

In [ ]:
from transformers import TrainingArguments

output_dir = "./results"
per_device_train_batch_size = 4
gradient_accumulation_steps = 4
optim = "paged_adamw_32bit"
save_steps = 100
logging_steps = 10
learning_rate = 2e-4
max_grad_norm = 0.3
max_steps = 500
warmup_ratio = 0.03
lr_scheduler_type = "constant"

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=True,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
)

Then finally pass everthing to the trainer

In [ ]:
from trl import SFTTrainer

max_seq_length = 512

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="instruction",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
)

AttributeError: ignored

We will also pre-process the model by upcasting the layer norms in float 32 for more stable training

In [ ]:
for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.float32)

NameError: ignored

## Train the model

Now let's train the model! Simply call `trainer.train()`

In [ ]:
trainer.train()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
10,2.615000
20,6.159800
30,31.242000
40,27.656800
50,8.083900
60,84.083900
70,42.182800
80,31.210600
90,52.311200
100,49.388800


Step,Training Loss
10,2.615000
20,6.159800
30,31.242000
40,27.656800
50,8.083900
60,84.083900
70,42.182800
80,31.210600
90,52.311200
100,49.388800


TrainOutput(global_step=500, training_loss=44.02412420654297, metrics={'train_runtime': 11189.3876, 'train_samples_per_second': 0.715, 'train_steps_per_second': 0.045, 'total_flos': 6.86950653272064e+16, 'train_loss': 44.02412420654297, 'epoch': 66.67})

During training, the model should converge nicely as follows:

![image](https://huggingface.co/datasets/trl-internal-testing/example-images/resolve/main/images/loss-falcon-7b.png)

The `SFTTrainer` also takes care of properly saving only the adapters during training instead of saving the entire model.

In [ ]:
model_to_save = trainer.model.module if hasattr(trainer.model, 'module') else trainer.model  # Take care of distributed/parallel training
model_to_save.save_pretrained("outputs")

In [ ]:
lora_config = LoraConfig.from_pretrained('outputs')
model = get_peft_model(model, lora_config)

In [ ]:
issues_dataset['text']

['### Human:Bebeğin doğumunun kırkıncı gününde dinen yapılması gereken işlemler var mıdır? ###Assistant:Bebeğin doğumunun kırkıncı gününe dair dinen yapılması gereken herhangi bir işlem bulunmamaktadır. Kırk gün boyunca bebeğin ve annenin evden dışarı çıkmaması, bebeğin kırkının çıkarılması gibi uygulama ve inanışların dinî bir dayanağı yoktur.',
 '### Human:Bebeğin göbek bağının cami, okul bahçesi gibi yerlere gömülmesi doğru mudur? ###Assistant:Göbek bağının cami, okul gibi mekânlara gömülmesine dair inanışın dinî bir dayanağı bulunmamaktadır. Ancak insanın bir parçası olduğu için saygı gereği göbek bağının uygun bir yere gömülmesi tavsiye edilir.',
 '### Human:Lohusanın ilk sütünün bebeğe verilmemesi, kırk gün dolmadan evden çıkmaması, lohusa kadınların bir araya gelmemesi gibi anlayışların dinî bir dayanağı var mıdır? ###Assistant:Lohusanın ilk sütünün bebeğe verilmemesi gerektiği şeklindeki anlayış, dinî bir dayanağa sahip olmadığı gibi tıbbi bilgilerle de uyuşmamaktadır.Lohusanın

# **Diyanet ile eğitilmiş**

In [ ]:
text = "Yeni doğan bebeğin sünnet ettirilmesi doğru mudur? "
device = "cuda:0"

inputs = tokenizer(text, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=100)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


Yeni doğan bebeğin sünnet ettirilmesi doğru mudur? İlk sünnetlerinin yapılması için çok önemli bir önceki önceki önceki önceki önceki önceki önceki önceki önceki önceki önceki önceki önceki önceki önceki önceki önceki önceki ön


In [ ]:
text = "Çocuğa isim koyarken nelere dikkat edilmelidir? "
device = "cuda:0"

inputs = tokenizer(text, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=50)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Çocuğa isim koyarken nelere dikkat edilmelidir? Çocuklarının çocukluklarının çocukluklarının çocukluklarının çocuklukların


In [ ]:
from huggingface_hub import login
login()

In [ ]:
model.push_to_hub("llama2-qlora-finetunined-french")

adapter_model.bin:   0%|          | 0.00/134M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/1littlecoder/llama2-qlora-finetunined-french/commit/bd58e6d3e94cfae0c9f404a35a1b8be55eeec81d', commit_message='Upload model', commit_description='', oid='bd58e6d3e94cfae0c9f404a35a1b8be55eeec81d', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
text = "Yeni doğan bebeğin sünnet ettirilmesi doğru mudur? "
device = "cuda:0"

inputs = tokenizer(text, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=100)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Yeni doğan bebeğin sünnet ettirilmesi doğru mudur? İlk sünnetlerinin yapılması için çok önemli bir önceki önceki önceki önceki önceki önceki önceki önceki önceki önceki önceki önceki önceki önceki önceki önceki önceki önceki ön


In [ ]:
text = "At eti helal midir?"
device = "cuda:0"

inputs = tokenizer(text, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=50)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1411: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


At eti helal midir?
 Hinweis: Die Videospiel-Kategorie ist nicht vollständig.
Dieser Artikel wurde zuletzt am 20. Februar 2019 um 15:00 Uhr aktualis


In [ ]:
text = "Piyango, Toto, Loto, İddia vb. Şans Oyunları Oynamanın Dini Hükmü?"
device = "cuda:0"

inputs = tokenizer(text, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=100)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Piyango, Toto, Loto, İddia vb. Şans Oyunları Oynamanın Dini Hükmü?
Piyango, Toto, Loto, İddia vb. Şans Oyunları Oynamanın Dini Hükmü? Piyango, Toto, Loto, İddia vb. Şans Oyunları Oynamanın Dini Hükmü?
Piyango, Toto, Loto, İddia vb. Şans Oyunları Oynam


In [ ]:
text = "Çocuğa isim koyarken nelere dikkat edilmelidir? "
device = "cuda:0"

inputs = tokenizer(text, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=50)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Çocuğa isim koyarken nelere dikkat edilmelidir? Çocuklarının çocukluklarının çocukluklarının çocukluklarının çocuklukların
